In [6]:
import json 
import pandas as pd
from urllib.request import urlopen
from disCap import finmod as FM
from disCap import disCap
import matplotlib.pyplot as plt
import numpy as np

'''
financialmodelingprep ask that the API key not be shared.  I keep mine in a json
file.  The API key is available and free for use provided by registration at 
the site, it is to not be shared or left in git. 
'''

with open(r"C:\Users\dgspr\.ssh\finmod_api_key.json") as f:
  api_key = json.load(f)['api_key']

print(api_key)

6163425f2c630fd89ae0c3b7a9fdbe52


In [2]:
symbols = ['ETHUSD','TMUS']
dfx = FM.portfolio_data(symbols, api_key)
dfx.head(10)

Reading ETHUSD and writing to archive
Symbol: ETHUSD.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading TMUS and writing to archive
Symbol: TMUS.  Keys: dict_keys(['symbol', 'historical'])
continue...



,symbol,close,prev_close,maxmed10,maxmed30,maxmed60,maxmed120,maxmed240
0,ETHUSD,1844.2606,1842.82,1867.270,1897.72,1933.57,2013.98,1868.19
1,TMUS,136.2850,135.37,138.825,140.50,137.87,149.99,148.57


In [3]:
with open('data/stock_description.json') as f:
  stock_description = json.load(f)

with open('data/investments.json') as f:
  investments = json.load(f)

stock_description_df = FM.stockDescriptionAs_df(stock_description)
investments_df, todays_df = FM.investmentsAs_df(investments)

symbols = stock_description.keys()

dfx = FM.portfolio_data(symbols, api_key)
dfx['rho'] = dfx['close']/dfx['maxmed30']
dfx = dfx.sort_values(by=['rho'], ascending=False)

todays_df = todays_df.merge(dfx, how='left', on='symbol')
todays_df['value'] =  todays_df['close'] * todays_df['shares'] 
todays_df['gain'] =  todays_df['value']/todays_df['invested']

Reading AMD and writing to archive
Symbol: AMD.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading PYPL and writing to archive
Symbol: PYPL.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading OKTA and writing to archive
Symbol: OKTA.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading AMZN and writing to archive
Symbol: AMZN.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading APPN and writing to archive
Symbol: APPN.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading WORK and writing to archive
Symbol: WORK.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading ALB and writing to archive
Symbol: ALB.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading ADBE and writing to archive
Symbol: ADBE.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading ESTC and writing to archive
Symbol: ESTC.  Keys: dict_keys(['symbol', 'historical'])
continue...

Reading GOOGL and writing to archive
Symbol: GOOGL

In [4]:
total_invested, total_value = sum(todays_df.invested), sum(todays_df.value)
todays_df = todays_df.merge( stock_description_df, on='symbol',how='left')
todays_df['prc_value'] = 100* ( todays_df['value']/total_value )
todays_df['prc_invested'] = 100 * (todays_df['invested']/total_value )
todays_df['basis'] = todays_df['invested']/todays_df['shares']

todays_df['rho60'] = todays_df['close']/todays_df['maxmed60']
todays_df['rho120'] = todays_df['close']/todays_df['maxmed120']
todays_df = todays_df.sort_values(by=['rho120'])


Red at gain = 0.057381025605536325.
Changes linear to yellow at 1.
From 1 to 2.7869015395322085 is green to blue shift


Total Invested:			44341.23
Total Value:			40062.15
Average Gain %:			  0.90
Median Gain:			  0.89


With consideration of reinvestment.
Total Invested (Adjusted):	42770.21
Total Value:			40062.15
Average Gain %:			  0.94
Median Gain:			  0.89


In [5]:
fig1, ax1 = plt.subplots()
todays_df = todays_df.sort_values(by=['gain'])
# rgb_gain(df.gain,0.8,0.95,2)
red_point = min( min(todays_df.gain), 0.95)
blue_point = max( max(todays_df.gain), 3)
green_point = 1

red_point = min(todays_df.gain)
blue_point = max(todays_df.gain)
#green_point = np.median( [1.0] )

print('Red at gain = {}.\nChanges linear to yellow at {}.\nFrom {} to {} is green to blue shift'.\
      format(red_point, green_point, green_point, blue_point))

label_ = list( map( disCap.mklab, todays_df.symbol, \
                   todays_df.invested, todays_df.value, todays_df.gain, todays_df.shares) )
ax1.pie(todays_df.value, labels=label_, autopct='%1.1f%%', shadow=True, radius=7, startangle=90, \
        colors= disCap.rgb_gain(todays_df.gain,red_point,green_point,blue_point) )

plt.show()

# Sold:  Invested 1700.00
# Sold:  Value 3400.71
# All reinvested.  

print('\n\nTotal Invested:\t\t\t{0:6.2f}\nTotal Value:\t\t\t{1:6.2f}\nAverage Gain %:\t\t\t{2:6.2f}'.\
      format(total_invested, total_value, total_value/total_invested))
print('Median Gain:\t\t\t{0:6.2f}'.format(np.median(todays_df.gain)))

# with consideration of sold & reinvested
# Sold:  Invested 1700.00 + 1400 + 1300 + 251  ( + JMIA + LMND + okta)
# Sold:  Value 3400.71 + 1032.39 + 2167.40 + 310
# # Sold:  Nio, BABA, JD invested (751 + 2554 + 1903) 2,117.59 699.99  1,704.94 
# All reinvested. 

investment_adj = 3400.71 + 1032.39 + 2167.40 + 310 + 2117.59 +699.99 + 1704.94
investment_adj =  investment_adj - 1701.00 - 1401 - 1301 - 251 - 751 - 2554 - 1903
total_investment_adj = total_invested - investment_adj 
print('\n\nWith consideration of reinvestment.')
print('Total Invested (Adjusted):\t{0:6.2f}\nTotal Value:\t\t\t{1:6.2f}\nAverage Gain %:\t\t\t{2:6.2f}'.\
      format(total_investment_adj, total_value, total_value/total_investment_adj))
print('Median Gain:\t\t\t{0:6.2f}'.format(np.median(todays_df.gain)))



In [8]:
fields = ['symbol','gain','basis','close','invested','value','close','prev_close','shares','prc_invested','prc_value','first_investment','last_investment'\
         ,'maxmed30','rho']


todays_df['rho60'] = todays_df['close']/todays_df['maxmed60']
todays_df['rho120'] = todays_df['close']/todays_df['maxmed120']
todays_df = todays_df.sort_values(by=['value'])
todays_df[fields].head(40)

,symbol,gain,basis,close,invested,value,close,prev_close,shares,prc_invested,prc_value,first_investment,last_investment,maxmed30,rho
4,BTCUSD,1.439748,29439.286728,42385.148438,28.00,40.312939,42385.148438,41626.195313,0.000951,0.046447,0.066872,2021.01.02,2021.01.29,34045.261719,1.244965
10,MMM,1.272163,155.593228,197.940000,201.00,255.704830,197.940000,198.170000,1.291830,0.333424,0.424170,2020.07.07,2020.07.07,200.400000,0.987725
18,TMUS,1.360093,105.889790,144.020000,201.00,273.378764,144.020000,144.630000,1.898200,0.333424,0.453488,2020.07.07,2020.07.22,148.055000,0.972747
0,ADBE,1.385536,448.656609,621.630000,783.00,1084.874892,621.630000,621.700000,1.745210,1.298862,1.799621,2020.06.05,2021.01.05,606.135000,1.025564
6,CVS,1.399757,58.838778,82.360000,1001.00,1401.156912,82.360000,83.060000,17.012590,1.660487,2.324279,2020.10.16,2020.10.16,81.800003,1.006846
2,AMD,1.198330,88.614992,106.190000,1251.00,1499.110778,106.190000,102.950000,14.117250,2.075194,2.486767,2021.02.16,2021.03.09,92.105000,1.152923
19,TSM,0.956554,121.937693,116.640000,1704.00,1629.968184,116.640000,116.150000,13.974350,2.826643,2.703837,2021.02.18,2021.03.31,120.050000,0.971595
17,SQM,1.801950,26.360329,47.500000,1001.00,1803.752175,47.500000,48.980000,37.973730,1.660487,2.992115,2020.03.03,2020.03.11,49.010000,0.969190
14,RDFN,2.225539,26.317222,58.570000,851.00,1893.933577,58.570000,59.090000,32.336240,1.411663,3.141710,2020.03.05,2020.03.10,62.420000,0.938321
13,PYPL,1.006971,273.622662,275.530000,1928.00,1941.439486,275.530000,283.170000,7.046200,3.198221,3.220514,2021.02.15,2021.03.05,299.140000,0.921074


In [ ]:
len(todays_df['value'])

In [ ]:
dfx = todays_df

sector_df = dfx[['sector','invested','value']].groupby('sector').agg('sum').reset_index()
total = sum( sector_df['value'] )
sector_df['gain'] = sector_df['value']/sector_df['invested']
sector_df['percentage_value'] = 100.00*sector_df['value']/total

area_df = dfx[['area','invested','value']].groupby('area').agg('sum').reset_index()
total = sum( area_df['value'] )
area_df['gain'] = area_df['value']/area_df['invested']
area_df['percentage_value'] = 100*area_df['value']/total

In [ ]:
fig1, ax1 = plt.subplots()
df = sector_df
df = df.sort_values(by=['gain'])

label_ = list( map( disCap.mklab, df.sector, df.invested, df.value, df.gain, df.gain) )
ax1.pie(df.value, labels=label_, autopct='%1.1f%%', shadow=True, radius=7, startangle=90, \
        colors= disCap.rgb_gain(df.gain,red_point,green_point,blue_point) )

plt.show()

In [ ]:
fig1, ax1 = plt.subplots()
df = area_df
df = df.sort_values(by=['gain'])

red_point = min(df.gain)
blue_point = max(df.gain)
green_point = np.median( df.gain )
green_point = 1.0

label_ = list( map( disCap.mklab, df.area, df.invested, df.value, df.gain, df.gain) )
ax1.pie(df.value, labels=label_, autopct='%1.1f%%', shadow=True, radius=7, startangle=90, \
        colors= disCap.rgb_gain(df.gain,red_point,green_point,blue_point) )

plt.show()

In [ ]:
todays_df = todays_df.sort_values(by=['last_investment'])
todays_df[fields].head(40)